**Load Existing Model**

In [ ]:

import sys
import face_swap

params = {'dataset_a': 'faceA',
          'dataset_b': 'faceB',
              'enc_att':True,
              'dec_att':True,
              'disc_att' :False,
              'batch_size': 6,
              'workers': 16,
              'res': 128,
              'res_layers_p': [2,6,9,14,17,20,23,26],
              'res_layers_p_weight': [1, 1, 1,1,1,1,1,1],
              'lr': 2e-4,
              'disc_layers': 3,
              'beta1': .5,
              'beta2': .999,
              'disc_perceptual_weight': 2.5,
              'perceptual_weight': 2.5,
              'edge_weight':.1,
              'eye_weight':0,
              'recon_weight': 3.,
              'train_epoch': 201,
              'ids_test': [0, 100],
              'ids_train': [0, 2],
              'save_every': 5,
              'save_img_every': 1,
              'lr_drop_start': 0,
              'lr_drop_every': 40,
              'save_root': 'test_a'}

rev = face_swap.FaceSwap(params)

In [ ]:
# Replce the file name with a model you have trained
rev.load_state("./output/face_swap_5.json")


**Import Inference Files**

In [ ]:
from converter.video_converter import VideoConverter
from detector.face_detector import MTCNNFaceDetector
import numpy as np


############ FIX TO WORK WITH COLAB #####################
# save np.load
np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
#########################################################

fd = MTCNNFaceDetector()
vc = VideoConverter()
vc.set_face_detector(fd)
vc.set_gan_model(rev)

# go back to old np.load
np.load = np_load_old


**Set Options**

In [ ]:
options = {
    # ===== Fixed =====
    "use_smoothed_bbox": True,
    "use_kalman_filter": True,
    "use_auto_downscaling": False,
    "bbox_moving_avg_coef": 0.65,
    "min_face_area": 35 * 35,
    "IMAGE_SHAPE": (128, 128, 3),
    # ===== Tunable =====
    "kf_noise_coef": 1e-3,
    "use_color_correction": "adain",
    "detec_threshold": 0.8,
    "roi_coverage": 0.9,
    "enhance": 0.,
    "output_type": 3,
    "direction": "AtoB", # ==================== This line determines the transform direction ====================
}

In [ ]:
########### Replace these two lines with your files ##############
fn_source_video = 'videos/gump.mp4'
fn_target_video = 'videos/tom.mp4'

if options["direction"] == "AtoB":
    input_fn = fn_source_video
    output_fn = "output/A_to_B_test.mp4"
elif options["direction"] == "BtoA":
    input_fn = fn_target_video
    output_fn = "output/B_to_A_test.mp4"

# Set the duration based on the clip you chose
duration =  (140,180) # None or a non-negative float tuple: (start_sec, end_sec). Duration of input video to be converted

**Run Inference**

In [ ]:
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)
